In [1]:
import pandas as pd
import torch as t
import pickle
from utils import plot_boards_general
import einops

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


In [2]:
# TODO for only flipped and flipped, mien yours put all layers into a correct tensor and a not blank tensor
# TODO then visualize
correct = {}
not_blank = {}

directory = 'attn_approx_results'
for probe_dir in ['get_probes_flipped_yours_mine', 'get_probes_flipped']:
    correct_t_full = []
    not_blank_t_full = []
    for layer in range(1, 8):
        with open(f'{directory}/{probe_dir}/attn_approx_correct_count_L{layer}.pkl', 'rb') as f:
            correct_t = pickle.load(f)
        with open(f'{directory}/{probe_dir}/attn_approx_not_blank_count_L{layer}.pkl', 'rb') as f:
            not_blank_t = pickle.load(f)
        correct_t_full.append(correct_t)
        not_blank_t_full.append(not_blank_t)
    correct_t_full = t.stack(correct_t_full)
    correct_t_full = einops.rearrange(correct_t_full, 'l r c a p -> a l p r c')
    # turn nan to 0
    # correct_t_full[correct_t_full != correct_t_full] = 0
    correct[probe_dir] = correct_t_full
    not_blank_t_full = t.stack(not_blank_t_full)
    not_blank_t_full = einops.rearrange(not_blank_t_full, 'l r c a p -> a l p r c')
    # turn nan to 0
    # not_blank_t_full[not_blank_t_full == 0] = 1
    not_blank[probe_dir] = not_blank_t_full

In [3]:
not_blank['get_probes_flipped_yours_mine'][0, 1, 1, :, :]

tensor([[    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,  8529., 12620., 20803.,     0.,     0.,     0.],
        [    0.,     0., 29118., 50000., 50000., 50000.,     0.,     0.],
        [    0.,     0., 50000., 50000., 50000., 50000.,     0.,     0.],
        [    0.,     0.,  8401., 16930., 29261., 12553.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.]])

In [4]:
# TODO: Fix the NaN cases
# TODO: Save
# TODO: Write Thesis
probe_dir = 'get_probes_flipped_yours_mine'
use_attn_approx = False
correct_new = einops.reduce(correct[probe_dir], "a l p r c -> l a r c", "sum")
not_blank_new = einops.reduce(not_blank[probe_dir], "a l p r c -> l a r c", "sum")
boards = (correct_new / not_blank_new)
not_blank_new = not_blank[probe_dir].clone()
not_blank_new[not_blank_new == 0] = 1
'''plot_boards_general(
    x_labels=[f'L{layer}' for layer in range(1, 8)],
    y_labels=[f'Pos {pos}' for pos in range(0, 59)],
    boards = (correct[probe_dir] / not_blank_new)[int(use_attn_approx)],
    color_range="not symmetric"
)'''
'''
boards_small = (correct[probe_dir] / not_blank[probe_dir])[int(use_attn_approx)][:4, :4]
plot_boards_general(
    x_labels=[f'L{layer}' for layer in range(1, 5)],
    y_labels=[f'Pos {pos}' for pos in range(0, 4)],
    boards = boards_small,
    color_range="not symmetric"
)'''
'''plot_boards_general(
    x_labels=[f'L{layer}' for layer in range(1, 8)],
    y_labels=["Real Pattern", "Avg Pattern"],
    boards = boards,
    color_range="not symmetric"
)'''

'plot_boards_general(\n    x_labels=[f\'L{layer}\' for layer in range(1, 8)],\n    y_labels=["Real Pattern", "Avg Pattern"],\n    boards = boards,\n    color_range="not symmetric"\n)'

In [6]:
from utils import save_plotly

In [11]:
correct_pos_15 = correct[probe_dir][1, :, 15, :, :]
correct_pos_15_not_rim = correct_pos_15[:, 1:-1, 1:-1].sum(dim=(-2, -1))
correct_pos_15_rim = correct_pos_15.sum(dim=(-2, -1)) - correct_pos_15_not_rim
not_blank_pos_15 = not_blank[probe_dir][1, :, 15, :, :]
not_blank_pos_15_not_rim = not_blank_pos_15[:, 1:-1, 1:-1].sum(dim=(-2, -1))
not_blank_pos_15_rim = not_blank_pos_15.sum(dim=(-2, -1)) - not_blank_pos_15_not_rim
accuracy_pos_15_not_rim = correct_pos_15_not_rim / not_blank_pos_15_not_rim
accuracy_pos_15_rim = correct_pos_15_rim / not_blank_pos_15_rim
# plot a line graph with plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(1, 8)], y=accuracy_pos_15_not_rim, mode='lines+markers', name='Not Rim'))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(1, 8)], y=accuracy_pos_15_rim, mode='lines+markers', name='Rim'))
fig.update_layout(
    title='Accuracy of attention approximation at position 15 over the layers',
    xaxis_title='Layer',
    yaxis_title='Accuracy',
)
fig.show()
save_plotly(fig, 'accuracy_pos_15_over_layers')

In [13]:
correct_layer_2 = correct[probe_dir][1, 2, :, :, :]
correct_layer_2_not_rim = correct_layer_2[:, 1:-1, 1:-1].sum(dim=(-2, -1))
correct_layer_2_rim = correct_layer_2.sum(dim=(-2, -1)) - correct_layer_2_not_rim
not_blank_layer_2 = not_blank[probe_dir][1, 2, :, :, :]
not_blank_layer_2_not_rim = not_blank_layer_2[:, 1:-1, 1:-1].sum(dim=(-2, -1))
not_blank_layer_2_rim = not_blank_layer_2.sum(dim=(-2, -1)) - not_blank_layer_2_not_rim
accuracy_layer_2_not_rim = correct_layer_2_not_rim / not_blank_layer_2_not_rim
accuracy_layer_2_rim = correct_layer_2_rim / not_blank_layer_2_rim
# plot a line graph with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=[f'{pos}' for pos in range(0, 59)], y=accuracy_layer_2_not_rim, mode='lines+markers', name='Not Rim'))
fig.add_trace(go.Scatter(x=[f'{pos}' for pos in range(0, 59)], y=accuracy_layer_2_rim, mode='lines+markers', name='Rim'))
fig.update_layout(title='Accuracy of attention approximation at layer 2 over the positions',
                     xaxis_title='Sequence Position',
                     yaxis_title='Accuracy')
fig.show()
save_plotly(fig, 'accuracy_layer_2_over_positions')

In [80]:
# TODO: Still missing comparison of Flipped only and yours and mine ...
# correct[probe_dir][0].sum() / 
not_blank

KeyError: 0

In [81]:
avg_accuracy_no_approx = correct[probe_dir][0].sum() / not_blank[probe_dir][0].sum()
avg_accuracy_approx = correct[probe_dir][1].sum() / not_blank[probe_dir][1].sum()
print(f'Average Accuracy without Approximation: {avg_accuracy_no_approx}')
print(f'Average Accuracy with Approximation: {avg_accuracy_approx}')

Average Accuracy without Approximation: 0.8178505897521973
Average Accuracy with Approximation: 0.8039326071739197
